GRUPO: Carlos Tardón y Carlos Morán Grupo 9

# Práctica 2

Vamos a ver en la práctica los conceptos vistos en clase sobre Recomendadores.
- En la primera parte veremos algunos conceptos de la librería Pandas de python ejemplificados en lo que hemos llamado Recomendador Simple, que ordena y muestra las películas en el TOP de popularidad pero no realiza recomendaciones personalizadas, es decir, a todos los usuarios les recomienda la misma lista de películas (como en el TOP 10 de Netflix).
- En la segunda parte veremos un recomendador personalizado basado en contenidos y 
- En la tercera parte veremos los recomendadores por filtrado colaborativo usando la librería SURPRISE y haciendo hincapié en como evaluar los resultados de la recomendación. 


## Primera parte. Recomendacion simple por popularidad

Esta primera parte está inspirada en Rounak Banik, 2018 Recommender Systems in Python: Beginner Tutorial DE DATACAMP.COM
Vamos a utilizar los dos datasets de MovieLens que se pueden descargar de: 
https://www.kaggle.com/rounakbanik/the-movies-dataset

- The Full Dataset: Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

- The Small Dataset: Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

En esta primera parte solo tienes que ejecutar las celdas del tutorial y responder las preguntas del final. No se pide implementar nada extra

### Recomendador simple basado en popularidad

Vamos a realizar una version simple de un Recomendador que ofrece recomendaciones generalizadas a todos los usuarios según la popularidad de la película y el género. La idea básica detrás de este recomendador es que las películas que son más populares y más aclamadas por la crítica tendrán una mayor probabilidad de gustar a la audiencia promedio. Este modelo no da recomendaciones personalizadas en base a los gustos de un usuario concreto.

La implementación de este modelo muy sencilla. Basta ordenar las películas según las calificaciones y la popularidad y mostrar las mejores películas de nuestra lista. Como paso adicional, podemos pasar un argumento de género para filtrar y mostrar solo las mejores películas de un género en particular. 

Los pasos que vamos a realizar en esta primera versión son los siguientes:
 - Decidir cual es el criterio que nos permite ordenar las peliculas recomendadas. 
 - Calcular esa métrica para cada pelicula de la lista total de películas.
 - Ordenar las películas y mostrar las mejores.

Cargamos los datos del archivo csv de peliculas usando pandas.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Pandas es un paquete de Python que proporciona estructuras de datos tabulares con columnas de tipo heterogéneo con etiquetas en columnas y filas. Estas tablas se conocen como DataFrames. Un dataframe es una estructura de datos con dos dimensiones en la cual se puede guardar datos de distintos tipos (como caractéres, enteros, valores de punto flotante, factores y más) en columnas. Es similar a una hoja de cálculo o una tabla de SQL

In [3]:
# Importamos Pandas
import pandas as pd
import warnings; 
warnings.simplefilter('ignore')  # para evitar algunos warnings al cargar los datos.

# Load Movies Metadata in a dataframe
md = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

# se puede imprimir solo las primeras 3 filas para ver como son los datos.
#md.head(3)
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Utilizo las clasificaciones de The Movie Database (TMDb)  para crear nuestra tabla de películas principales. 
Vamos a utilizar la fórmula de calificación ponderada de IMDB Internet Movie Database.

Calificación ponderada o Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
dónde,
- v es el número de votos para la película
- m son los votos mínimos requeridos para aparecer en la tabla
- R es la calificación promedio de la película
- C es el voto medio

El siguiente paso es determinar un valor apropiado para m, los votos mínimos requeridos para aparecer en la tabla. Usaremos el percentil 95 como nuestro límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 95% de las películas de la lista.

Vamos a hacer una tabla Top 250 por popularidad y otra para seleccionar las películas de un genero en particular.  


In [4]:
from ast import literal_eval
import numpy as np
import matplotlib.pyplot as plt

In [5]:
## obtenemos la media de las puntuaciones de las películas de la tabla. 
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [6]:
# cuantas cumplen el percentil 0,95
m = vote_counts.quantile(0.95)
m

434.0

Para que una película sea elegida tiene que tener por los menos 434 votos en TMDB (95%). Tambien vemos que la media de rating de las peliculas en TMDB es de 5.244 en una escala de 10. Hay 2274 peliculas que cumplen estos criterios.

In [7]:
# Utilizo la columna release_date para construir una columna year, porque solo me interesa el año. 
# No es necesario comprender todos los detalles pero puedes consultar la documentación de pandas.

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
# limpiamos la lista de generos para que se quede solo con el nombre en vez de la estructura [{'id': 16, 'name': 'Animation'}. Sólo ejecutar una vez. 
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [9]:
md['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

Vamos a mostrar el resultado despues de modificar la columna genres y añadir la columna year para el año. 

In [10]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


In [11]:

seleccionadas = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
seleccionadas.shape
# shape devuelve el numero filas y columnas de la tabla de peliculas seleccionadas. 

(2274, 6)

In [12]:
seleccionadas.head(3)

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]"


In [13]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
seleccionadas['wr'] = seleccionadas.apply(weighted_rating, axis=1)

In [15]:
seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)

###  TOP 15 de peliculas

In [16]:
seleccionadas.head(15)
#mostramos el TOP 15

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


Vemos que tres películas de Christopher Nolan:  **Inception**, **The Dark Knight** e **Interstellar** son las preferidas por popularidad en la parte superior de nuestra lista de TOP 15. En la tabla se ve un fuerte sesgo de los usuarios de TMDB hacia géneros y directores particulares, pero de los sesgos no hablaremos en esta práctica. 

Vamos a construir una funcion que permite filtrar y construir una tabla de peliculas recomendadas para generos particulares. 

In [17]:
# Separamos las peliculas que tienen varios generos de forma que aparezcan varias filas de la misma película.
# una por cada genero.
# De nuevo no hace falta entender todos los detalles. Sólo observar el comportamiento en la tabla resultado.

s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [18]:
gen_md.head(3)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family


In [19]:
# Metemos todo el código anterior en una función.
# Vamos a relajar nuestras condiciones predeterminadas al percentil 85 en lugar de 95 para tener más generos.
    
def construye_tabla(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
    
    seleccionadas['wr'] = seleccionadas.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)
    
    return seleccionadas

Veamos nuestro método en acción mostrando las 15 mejores películas románticas. Con esto forzamos el genero que nos interesa y resolvemos el problema de que el romance casi no figuraba en nuestro Top  genérico a pesar de ser uno de los géneros de películas más populares.

### TOP Películas románticas

In [20]:
construye_tabla('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


Hasta aquí lo único que hemos hecho es ordenar el catalogo. Las recomendaciones son generales (no personalizadas) por popularidad, pero lo que nosotros queremos hacer son recomendaciones personalizadas para un usuario del que sabemos su perfil, sus preferencias, alguna película que le ha gustado.
La siguiente parte corresponde al recomendador basado en contenido que dada una película (puede ser usada como perfil) recomienda peliculas similares. En la tercera parte veremos el filtrado colaborativo.

Para el Recomendador simple hemos usado las películas del dataset completo, mientras que para el sistema de recomendación personalizado haremos uso del segundo data set (Small)

### Ejercicio parte 1.

Realiza algunas pruebas de recomendaciones por popularidad y comenta los resultados obtenidos.  Incluye los resultados y los comentarios en este archivo.

Nos vamos a centrar primero en el top general. Veamos que ocurre con un pequeño cambio en el percentil

In [52]:
#Función para mostrar el top de películas sin depender del género
def construye_tabla_general(percentile=0.85):
    vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
    
    seleccionadas['wr'] = seleccionadas.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)
    
    return seleccionadas

In [54]:
#Primero con percentile=0.95
construye_tabla_general(0.95).head(15)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.917588
12481,The Dark Knight,2008,12269,8,123.167259,7.905871
22879,Interstellar,2014,11187,8,32.213481,7.897107
2843,Fight Club,1999,9678,8,63.869599,7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.871787
292,Pulp Fiction,1994,8670,8,140.950236,7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.861927
351,Forrest Gump,1994,8147,8,48.307194,7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.851924


In [55]:
#Luego con el percentile por defecto
construye_tabla_general().head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.585574
15480,Inception,2010,14075,8,29.108149,7.984042
12481,The Dark Knight,2008,12269,8,123.167259,7.981708
22879,Interstellar,2014,11187,8,32.213481,7.979952
2843,Fight Club,1999,9678,8,63.869599,7.976853
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.974825
292,Pulp Fiction,1994,8670,8,140.950236,7.974187
314,The Shawshank Redemption,1994,8358,8,51.645403,7.973232
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.972807
351,Forrest Gump,1994,8147,8,48.307194,7.972546


En estas dos últimas celdas hemos comprobado una de las propiedades del Weighed Rating

Recordamos que obtenemos el Weighed Rating (wr) de la siguiente manera:

wr = $(\frac{v}{v + m} * R) + (\frac{m}{v + m} * C)$
dónde:
<ul>
<li> v es el número de votos para la película </li>
<li> m son los votos mínimos requeridos para aparecer en la tabla </li>
<li> R es la calificación promedio de la película </li>
<li> C es el voto medio </li>
</ul>

Vemos que esta definición es muy flexible ya que, depuendiendo de donde se haga el corte (el valor del percentil que escojamos). Cuanto mayor sea el percentil, mayor será el número de votos requerido y estaremos premiando a películas con más votos. Por otro lado, cuanto menor sea este valor, menor será el número mínimo de votos requerido y más oportunidades estaremos dando a películas con menos votos pero mejores valoraciones. En nuestro caso, esta situación podemos apreciarla al comparar el Top 15 para los valores del percentil $p=0,95$ y $p=0.85$. Como la distribución dada es lo suficientemente poco uniforme (solo el 5% de las películas tienen más de 434 votos pero varias tienen más de 10.000), vemos un gran cambio en la posición de películas como Dilwale Dulhania Le Jayenge, que tiene 661 votos con un promedio de 9 (mayor que el de Inception). Su wr correspondiente para $p=0,95$ es $wr\equiv7,58$, lejos del $wr=7,81$ de "Se7en" que le permitiría entrar en el top. Si bajamos $p=0,85$, el wr de esta película pasa a ser $wr=8,5$, poniéndola cómodamente como la nº1 de la lista.

Nos centramos ahora en los tops por géneros.

##### Parece que hay un claro sesgo hacia algunos géneros, por ejemplo si filtramos por el género Action, vemos que muchas de estas películas están incluidas en el top 15 general

In [24]:
construye_tabla('Action').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.955099
12481,The Dark Knight,2008,12269,8,123.167259,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.918382
256,Star Wars,1977,6778,8,42.149697,7.908327
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.896841
4135,Scarface,1983,3017,8,11.299673,7.802046
9430,Oldboy,2003,2000,8,10.616859,7.711649
1910,Seven Samurai,1954,892,8,15.01777,7.426145


En la siguiente celda vemos otro de los problemas de este recomendador. Al dar la misma recomendación sobre una película (independientemente del género dominante de la misma), al clasificar por un género secundario obtendremos películas que están mejor valoradas por pertenecer a un género predominante. Por ejemplo, al clasificar por comedy, obtenemos películas como Forrest Gump, o Back to the Future, que si bien tienen tintes de comedia, están bien posicionadas por otro tipo de factores. Esto puede ser un problema (por ejemplo, si una persona quiere ver una película que sea puramente de comedia), pero también puede ser algo beneficioso, como comentaremos más adelante

In [25]:
construye_tabla('Comedy').head(10)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.463024
351,Forrest Gump,1994,8147,8,48.307194,7.963363
1225,Back to the Future,1985,6239,8,25.778509,7.952358
18465,The Intouchables,2011,5410,8,16.086919,7.945207
22841,The Grand Budapest Hotel,2014,4644,8,14.442048,7.936384
2211,Life Is Beautiful,1997,3643,8,39.39497,7.919430
732,Dr. Strangelove or: How I Learned to Stop Worr...,1964,1472,8,9.80398,7.809073
3342,Modern Times,1936,881,8,8.159556,7.695554
883,Some Like It Hot,1959,835,8,11.845107,7.680781
1236,The Great Dictator,1940,756,8,9.241748,7.651762


#### ¿Te parece útil esta forma de recomendación de contenidos? 

Vamos a ver las diversas formas de las que podemos aprovechar este recomendador. La primera es la siguiente:
tenemos a una persona dentro del grupo (su nombre empieza por Carlos), que siempre se encuentra indeciso a la hora de elegir película. El problema es que no podemos usar una recomendación personalizada, pues todavía no ha decidido qué películas le gustan(hace mucho que no va al cine). Entonces, podemos usar el recomendador de la siguiente forma: cada día, Carlos elige un género que le apetezca, y de entre las películas mejor valoradas de ese género, elige una cuyo nombre le atraiga. Después de verla, tendrá que darle una puntuación y decidir si le gusta esa película o no. De esa forma, como estas películas están mejor valoradas por popularidad, es más probable que los datos que Carlos recoja sean más fiables a la hora de usar un recomendador basado en contenido. Así, va creando una pequeña base de datos de películas que le gustan, y cuando esta sea lo suficientemente amplia, podrá usar esos datos para emplear un recomendador basado en contenidos. Por ejemplo, hoy le apetece ciencia ficción, así que hace la siguiente búsqueda:

In [26]:
construye_tabla('Science Fiction').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.939069
22879,Interstellar,2014,11187,8,32.213481,7.923728
256,Star Wars,1977,6778,8,42.149697,7.876106
1225,Back to the Future,1985,6239,8,25.778509,7.865868
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.860722
1163,A Clockwork Orange,1971,3432,8,17.112594,7.764533
1901,Metropolis,1927,666,8,14.487867,7.078592
14551,Avatar,2009,12114,7,185.070892,6.952299
17818,The Avengers,2012,12000,7,89.887648,6.951856
23753,Guardians of the Galaxy,2014,10014,7,53.291601,6.942571


Supongamos que Carlos elige ver Inception, y esta le encanta, así que decide ver otras películas de este mismo género, y descubre que le apasionan. Esto otorga una gran información, pues no solo hemos descubierto las películas que le gustan, sino que también tenemos un género bien valorado por él, y todo esto pese a la sencillez del método de recomendación simple. Estos datos otorgan un buen punto de partida para ejecutar una recomendación basada en contenido.

La segunda aplicación de este recomendador sería útil si quisieramos otorgar premios a películas, un premio por cada género. Ejecutaríamos una recomendación por género,( y filtraríamos por año en el caso de que quisieramos premiar las películas estrenadas en el año actual). Después otorgaríamos un premio a la primera (o primeras) películas de cada género.

#### ¿Sería útil también para otras aplicaciones, por ejemplo, para recomendar libros o restaurantes?
En el caso de los libros, este recomendador sería útil de las mismas formas que hemos señalado anteriormente, si bien en el caso de los libros influye mucho(tal vez más incluso que el las películas) el autor, y el estilo personal del mismo. Por tanto sería más recomendable usar un recomendador colaborativo, pues en este caso influye mucho el estilo, y además los gustos de los libros se transmiten fundamentalmente boca a boca(en el caso de las películas también hay publicidad y anuncios), lo que hace que, si a una persona le gustan los mismos libros que a mi, es muy probable que las recomendaciones colaborativas sean muy efectivas.

En el caso de los restaurantes, si bien influyen los gustos personales en cuanto a tipo de comida, es más decisiva la situación geográfica de cada restaurante concreto (al menos en situaciones sociales informales como quedadas de amigos). Por tanto, el recomendador simple nos ayudaría bastante bien a elegir los restaurantes mejor valorados filtrando por zona geográfica, en el caso de que estemos indecisos en cuanto a gustos culinarios.

## Segunda parte.  Recomendacion basada en contenido


El recomendador que construimos en la sección anterior da la misma recomendación a todos, independientemente del gusto personal del usuario. Si una persona que le gustan las películas románticas (y odia la acción) mirara nuestro Top 15, probablemente no le gustarían la mayoría de las películas. Si esta persona mira nuestras listas por género, no estaría recibiendo las mejores recomendaciones.

Por ejemplo, considere a una persona que le gustan: "Dilwale Dulhania Le Jayenge", "My Name is Khan" y "Kabhi Khushi Kabhi Gham". Si miramos los datos de estas películas podríamos inferir que a esta persona le gusta el actor Shahrukh Khan y el director Karan Johar. Incluso si él / ella tuviera acceso a la tabla de películas románticas no las encontraría como las principales recomendaciones, porque no son películas populares. 

Para personalizar más nuestras recomendaciones, voy a crear un motor que calcule la similitud entre películas en función de ciertas métricas y sugiera películas que son más similares a una película en particular que le gustó a un usuario. Como hemos visto en teoría este tipo de recomendaciones se conocen como **recomendación o filtrado basado en contenido**.

Vamos a ver cómo construir dos recomendadores basados en contenido:

- Usando los resúmenes textuales de las películas y taglines. Los taglines son las frases cortas que  suelen aparecer en el cartel de la película, acompañando al título y a una imagen destacada y representativa de la historia que presenta.
- Usando el reparto de películas, equipo, palabras clave y género



In [27]:
# usaremos el conjunto pequeño de películas por eficiencia. 
# Leemos los datos y limpiamos algunos valores nulos.
links_small = pd.read_csv('./data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
## selecciono las peliculas de links_small con los id de la tabla de metadatos 
smd = md[md['id'].isin(links_small)]
smd.shape
## shape devuelve el numero de filas y columnas. 
## (9099, 25)
# Tenemos *9099* películas disponibles en nuestro conjunto de datos de metadatos de películas pequeñas, que es 5 veces más pequeño que nuestro conjunto de datos original de 45000 películas.

(9099, 25)

In [28]:
smd.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


Tenemos que medir la similitud de las películas con las preferencias del usuario. En este caso usaremos como preferencia una pelicula que le haya gustado antes para recuperar (por similitud) otras películas parecidas.  Elegimos usar similitud usando el atributo de descripción (textual) y el lema (tambien textual). Para calcular la similitud entre dos textos usamos el modelo del espacio vectorial.

In [29]:
smd['tagline'] = smd['tagline'].fillna('')
#El método fillna permite sustituir los valores nulos de una estructura pandas

smd['description'] = smd['overview'] + smd['tagline']
# creamos un campo llamado descrition con la 
smd['description'] = smd['description'].fillna('')

In [30]:
smd.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...


In [31]:
print(smd['description'])

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite Pokémon characters are back,...
45265    While holidaying in the French Alps, a Swedish...
Name: description, Length: 9099, dtype: object


#### Similitud del coseno 

Utilizamos el vectorizador TF-IDF
Para calcular un valor de similitud se puede usar ** cosine_similarity** o ** linear_kernel ** de sklearn. La segunda es más eficiente. 
https://scikit-learn.org/stable/modules/metrics.html

La medida de similitud del coseno calcula un valor numerico que representa la similitud entre dos vectores de términos. En este caso los vectores representan los textos de la descripción normalizados.

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [33]:
tfidf_matrix.shape
# filas, columnas

(9099, 268124)

In [34]:
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [35]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

Ahora tenemos una matriz de similitud de coseno por pares para todas las películas en nuestro conjunto de datos. El siguiente paso es escribir una función que devuelva las 30 películas más similares según la puntuación de similitud del coseno.

In [36]:
smd = smd.reset_index() 
#reset_index resets the index column) to a regular column and uses a numeric sequence as new index.

titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
#Un objeto "Series" es un vector con datos indexados

In [37]:
indices.head(3)

title
Toy Story           0
Jumanji             1
Grumpier Old Men    2
dtype: int64

In [38]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

###  Ejercicio parte 2: 
Prueba algunas consultas y comenta los resultados. Compara los resultados con el tipo de recomendaciones realizadas en la parte 1. 
    
Empezamos probando con una de las películas favoritas de CarlosT: El Señor de los Anillos: el retorno del Rey

In [39]:
get_recommendations("The Lord of the Rings: The Return of the King").head(10)

4388                The Lord of the Rings: The Two Towers
1673                                The Lord of the Rings
3865    The Lord of the Rings: The Fellowship of the Ring
2056                                        The Mod Squad
8723            The Hobbit: The Battle of the Five Armies
7257               Harry Potter and the Half-Blood Prince
6718                                         Transformers
2761                       National Lampoon's Last Resort
6041                                          Night Watch
8431                  The Hobbit: The Desolation of Smaug
Name: title, dtype: object

Parece que funciona bastante bien, pues como primeros resultados nos recomienda las otras películas de la misma trilogía. Además, entre estas 10 primeras recomendaciones también se encuentran películas de la saga del Hobbit, que pertenece al mismo universo ficticio, y cuyos libros fueron escritos por el mismo autor.   
Además, nos recomienda  una película de Harry Potter, que si bien no está ambientada en la Edad Media, al igual que El Señor de los Anillos contiene elementos fantásticos, y podría interesar al mismo público objetivo, si bien es cierto que Harry Potter está dirigido a un público más joven.     
Por último, hay que destacar un fallo que ha tenido el recomendador, pues en esa lista aparecen películas como Transformers o National Lampoon's Last Resort, que son películas muy diferentes al señor de los anillos, y que a CarlosT, como seguidor de la saga de Tolkien, no le gustan demasiado. Vamos a ver por qué nos ha recomendado estas películas.

Sabemos que las recomendaciones que hemos obtenido se basan en similitudes entre películas que tienen un campo 'description' parecido, así que veamos la descripción de las películas que nos ha proporcionado el recomendador.

In [40]:
smd[smd['original_title']=="The Lord of the Rings: The Return of the King"]["description"].iloc[0]

"Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor from Sauron's forces. Meanwhile, Frodo and Sam bring the ring closer to the heart of Mordor, the dark lord's realm.The eye of the enemy is moving."

In [41]:
# Vemos que esta primera recomendación también contiene las palabras Frodo,Sam,Aragorn,Mordor, entre otras, que
# nos dan mucha información de la similitud entre esta película y la anterior, debido a la baja frecuencia de 
# las mismas en la totalidad de descripciones consideradas. Esto las convierte en palabras clave
smd[smd['original_title']=="The Lord of the Rings: The Two Towers"]["description"].iloc[0]

'Frodo and Sam are trekking to Mordor to destroy the One Ring of Power while Gimli, Legolas and Aragorn search for the orc-captured Merry and Pippin. All along, nefarious wizard Saruman awaits the Fellowship members at the Orthanc Tower in Isengard.A New Power Is Rising.'

In [42]:
# Vemos que también contiene las palabras Fellowship, Ring, y Sauron
smd[smd['original_title']=="The Lord of the Rings"]["description"].iloc[0]

"The Fellowship of the Ring embark on a journey to destroy the One Ring and end Sauron's reign over Middle-earth.Fantasy...beyond your imagination"

In [43]:
# La película del hobbit, al poseer la palabra Sauron produce una alta similitud con la búsqueda original,
# pues muy pocas películas la poseen
smd[smd['original_title']=="The Hobbit: The Battle of the Five Armies"]["description"].iloc[0]

"Immediately after the events of The Desolation of Smaug, Bilbo and the dwarves try to defend Erebor's mountain of treasure from others who claim it: the men of the ruined Laketown and the elves of Mirkwood. Meanwhile an army of Orcs led by Azog the Defiler is marching on Erebor, fueled by the rise of the dark lord Sauron. Dwarves, elves and men must unite, and the hope for Middle-Earth falls into Bilbo's hands.Witness the defining chapter of the Middle-Earth saga"

In [44]:
# Veamos por qué nos ha recomendado las películas Transformers y  National Lampoon's Last Resort
smd[smd['original_title']=="Transformers"]["description"].iloc[0]

'Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it.Their war. Our world.'

In [45]:
smd[smd['original_title']=="National Lampoon's Last Resort"]["description"].iloc[0]

"Sam and Dave are living the boring life until they are beckoned to Sam's uncle's Island. When they get there (still not quite sure how that worked) they are compelled by beautiful women and a dastardly enemy of the Island. After accidentally convincing Sam's uncle to sign away rights to his island, they must somehow fix the problem."

Como podemos observar,el nombre "Sam" aparece tanto en la descripción de la película sobre la que ejecutamos la búsqueda, como en estas dos últimas películas, sobre las que (consideramos) el recomendador se equivocó. Esto solo puede significar una cosa: la palabra Sam aparece en la descripción de muy pocas películas, y al aparecer en más de una ocasión en la descripción de "Transformers" y de "Last Resort", el recomendador consideró que había una gran similitud entre estas películas.   

Aquí vemos un gran fallo de este recomendador en concreto, y también una dificultad a la hora de implementar un recomendador basado en contenido, pues las elecciones tomadas para representar el contenido de una película concreta influyen mucho en el resultado final que obtendremos, y es por ello que hay que intentar elegir representaciones que identiquen el contenido de la película en concreto. En este caso es difícil, pues es complicado discernir entre el nombre Sam, un nombre bastante común(que no habría que incluir en la descripción, o por lo menos su peso tendría que ser menor), y el nombre Frodo, que identifica muy bien la trilogía del Señor de los Anillos.

Comparemos este recomendador con el obtenido en la parte 1. Probemos el caso de Interstellar, una película que está catalogada como perteneciente a los géneros Adventure, Drama, Science Fiction:

In [46]:
get_recommendations("Interstellar").head(10)

4208       Suburban Commando
1312                 Gattaca
4442     Final Destination 2
3056           Space Cowboys
7669         All Good Things
282                 Stargate
8032              Prometheus
2631    They Might Be Giants
1260                   Spawn
1329       Starship Troopers
Name: title, dtype: object

In [47]:
construye_tabla('Adventure').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.906526
22879,Interstellar,2014,11187,8,32.213481,7.883426
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.854939
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.843867
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.832647
256,Star Wars,1977,6778,8,42.149697,7.812801
1225,Back to the Future,1985,6239,8,25.778509,7.797828
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.790329
5481,Spirited Away,2001,3968,8,41.048867,7.695056
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.465435


In [48]:
construye_tabla('Drama').head(10)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.607205
12481,The Dark Knight,2008,12269,8,123.167259,7.983111
22879,Interstellar,2014,11187,8,32.213481,7.981489
2843,Fight Club,1999,9678,8,63.869599,7.978628
314,The Shawshank Redemption,1994,8358,8,51.645403,7.975286
351,Forrest Gump,1994,8147,8,48.307194,7.974653
834,The Godfather,1972,6024,8,41.109264,7.965843
24860,The Imitation Game,2014,5895,8,31.59594,7.965106
359,The Lion King,1994,5520,8,21.605761,7.962771
18465,The Intouchables,2011,5410,8,16.086919,7.962025


In [49]:
construye_tabla('Science Fiction').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.939069
22879,Interstellar,2014,11187,8,32.213481,7.923728
256,Star Wars,1977,6778,8,42.149697,7.876106
1225,Back to the Future,1985,6239,8,25.778509,7.865868
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.860722
1163,A Clockwork Orange,1971,3432,8,17.112594,7.764533
1901,Metropolis,1927,666,8,14.487867,7.078592
14551,Avatar,2009,12114,7,185.070892,6.952299
17818,The Avengers,2012,12000,7,89.887648,6.951856
23753,Guardians of the Galaxy,2014,10014,7,53.291601,6.942571


Como vemos, en el recomendador basado en contenido hay mucha variabilidad. Nos ha recomendado películas de comedia y ciencia ficción como Suburban Commando, películas de ciencia ficción pura como Starship Troopers, y también películas de ciencia ficción y drama como Gattaca. Esta variabilidad nos asegura que el usuario puede obtener distintas opciones más enfocadas a su contenido preferido, pero diferentes entre sí, de tal manera que sea él quien decida qué elige ver.  

Además hay que destacar que estas películas no aparecen en ningún caso en las recomendaciones por popularidad, lo que permite que el usuario explore nuevas películas que tal vez son menos conocidas. Esto se traduce en beneficios potencialmente más elevados para la industria del cine de forma agregada, pues si solo listamos las películas más relevantes habría menor financiación para películas de menor presupuesto, lo que produciría que solo los grandes proyectos salieran adelante. Esto, a su vez, haría que hubiera menor demanda de películas, pues el cliente se aburriría viendo constantemente las películas más relevantes.

Como punto a favor del sistema basado en popularidad, podemos destacar que los resultados obtenidos por este método siempre van a ser fiables, es decir, siempre van a ser películas de calidad. En el caso de Interestellar, en cualquiera de sus géneros obtenemos recomendaciones como The Avengers, Avatar o Star Wars, que cuentan con enormes presupuestos que han permitido unas calidades visuales mucho mejores que las de las películas que nos devuelve el recomendador basado en contenido. Esta cualidad podría no ser usada mucho por los usuarios, pero desde luego es una información a tener en cuenta por las diversas empresas cinematográficas: las empresas inversoras, las diversas academias encargadas de otorgar premios a directores y actores, etc.